### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [54]:
# Create output for "District Summary"
total_schools = len(school_data["school_name"])
total_students = school_data["size"].sum()
total_budget = school_data["budget"].sum()
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()
passing_math_scores = (((student_data.loc[student_data["math_score"] >= 70, "math_score"]).count())
                       /total_students)*100
passing_reading_scores = (((student_data.loc[student_data["reading_score"] >= 70, "reading_score"]).count())
                          /total_students)*100
overall_pass_rate = (passing_math_scores + passing_reading_scores)/2
# Create a DataFrame of frames using a dictionary of lists for "District Summary"
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [passing_math_scores],
    "% Passing Reading": [passing_reading_scores],
    "% Overall Passing Rate": [overall_pass_rate]})
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [55]:
# Create output for "School Summary"
# Drop column :School ID" from dataframe school_data and name school_summary
school_summary = school_data.drop(columns=["School ID"])
# Rename columns
school_summary = school_summary.rename(columns = {"school_name":"School Name", "type":"School Type",
                                                  "size":"Total Students", "budget":"Total School Budget"})
# Create new columns
per_student_budget = school_summary["Total School Budget"]/school_summary["Total Students"]
school_summary["Per Student Budget"] = per_student_budget
# Sort school_summary alphabetically
school_summary_sort = school_summary.sort_values("School Name")
# Re-index school_summary_sort
school_summary_sort = school_summary_sort.reset_index(drop=True)

# Create dataframe which is a subset of dataframe school_data to calculate average math and reading scores 
math_reading = student_data.loc[:, ["school_name", "reading_score", "math_score"]]
# Use for % math and reading >= 70
math_percent = math_reading.drop(columns=["reading_score"])
reading_percent = math_reading.drop(columns=["math_score"])
# Extract rows of score >= 70
math_percent_ext = math_percent[math_percent["math_score"] >= 70]
reading_percent_ext = reading_percent[reading_percent["reading_score"] >= 70]
# Group each by school name and count
math_per_gb = math_percent_ext.groupby(by="school_name", as_index=False)
reading_per_gb = reading_percent_ext.groupby(by="school_name", as_index=False)
math_per_gb_c = math_per_gb.count()
reading_per_gb_c = reading_per_gb.count()

# Append groupby's and drop school name columns
math_reading_percent_raw = pd.concat([math_per_gb_c,reading_per_gb_c], axis=1)
math_reading_percent_raw = math_reading_percent_raw.drop(columns=["school_name"])
# Rename columns
math_reading_percent_raw = math_reading_percent_raw.rename(columns = {"math_score":"Math >= 70", 
                                                                      "reading_score":"Reading >= 70"})
per_math_school = (math_reading_percent_raw["Math >= 70"]*100)/school_summary_sort["Total Students"]
per_reading_school = (math_reading_percent_raw["Reading >= 70"]*100)/school_summary_sort["Total Students"]

# math_reading_percent_raw.head()

math_reading_gb = math_reading.groupby(by="school_name", as_index=False)
# math_reading_gb.head()
math_reading_mean = math_reading_gb.mean()

math_reading_mean = math_reading_mean.drop(columns=["school_name"])
# Append math_reading_mean to school_summary_sort
comb_ss_mr = pd.concat([school_summary_sort,math_reading_mean], axis=1)
comb_ss_mr["% Passing Math"] = per_math_school
comb_ss_mr["% Passing Reading"] = per_reading_school
comb_ss_mr["Overall Passing Rate"] = (per_math_school + per_reading_school)/2
comb_ss_mr = comb_ss_mr.rename(columns = {"reading_score":"Average Reading Score", "math_score":"Average Math Score"})
comb_ss_mr["Total School Budget"] = comb_ss_mr["Total School Budget"].map("${:,.2f}".format)
comb_ss_mr["Per Student Budget"] = comb_ss_mr["Per Student Budget"].map("${:,.2f}".format)
comb_ss_mr["Total Students"] = comb_ss_mr["Total Students"].map("{:,}".format)
comb_ss_mr

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,81.033963,77.048432,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,95.586652
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,73.363852
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,80.746258,77.102592,68.309602,79.299014,73.804308
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,95.265668
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.934412,77.289752,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.505855,96.252927,94.379391
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,65.683922,81.316421,73.500171
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.966394,77.072464,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

# * Sort and display the top five schools in overall passing rate

In [56]:
comb_ss_mr_top = comb_ss_mr.sort_values(by=["Overall Passing Rate"], inplace=False, ascending=False)
comb_ss_mr_top.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,95.586652
12,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.848930,83.418349,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.594595,95.945946,95.270270
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,95.265668
13,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.989488,83.274201,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [57]:
comb_ss_mr_top.tail(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,80.746258,77.102592,68.309602,79.299014,73.804308
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.966394,77.072464,66.057551,81.222432,73.639992
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,65.683922,81.316421,73.500171
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,73.363852
10,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,80.744686,76.842711,66.366592,80.220055,73.293323


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [70]:
# Create working dataframe from student_data with columns for school name and grade
dfm9 = student_data.loc[student_data["grade"] =="9th", ["school_name", "grade", "math_score"]]
dfm10 = student_data.loc[student_data["grade"] =="10th", ["school_name", "grade", "math_score"]]
dfm11 = student_data.loc[student_data["grade"] =="11th", ["school_name", "grade", "math_score"]]
dfm12 = student_data.loc[student_data["grade"] =="12th", ["school_name", "grade", "math_score"]]
dfm9 = dfm9.groupby(by="school_name", as_index=False)
dfm9 = dfm9.mean()
dfm9 = dfm9.rename(columns = {"math_score":"9th Grade Math Score"})
dfm9 = dfm9.rename(columns = {"school_name":"School Name"})

dfm10 = dfm10.groupby(by="school_name", as_index=False)
dfm10 = dfm10.mean()
dfm10 = dfm10.drop(["school_name"], axis=1)
dfm10 = dfm10.rename(columns = {"math_score":"10th Grade Math Score"})

dfm11 = dfm11.groupby(by="school_name", as_index=False)
dfm11 = dfm11.mean()
dfm11 = dfm11.drop(["school_name"], axis=1)
dfm11 = dfm11.rename(columns = {"math_score":"11th Grade Math Score"})

dfm12 = dfm12.groupby(by="school_name", as_index=False)
dfm12 = dfm12.mean()
dfm12 = dfm12.drop(["school_name"], axis=1)
dfm12 = dfm12.rename(columns = {"math_score":"12th Grade Math Score"})

m_s_b_grade = pd.concat([dfm9, dfm10, dfm11, dfm12], axis=1)
m_s_b_grade

,School Name,9th Grade Math Score,10th Grade Math Score,11th Grade Math Score,12th Grade Math Score
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [71]:
# Create working dataframe from student_data with columns for school name and grade
dfr9 = student_data.loc[student_data["grade"] =="9th", ["school_name", "grade", "reading_score"]]
dfr10 = student_data.loc[student_data["grade"] =="10th", ["school_name", "grade", "reading_score"]]
dfr11 = student_data.loc[student_data["grade"] =="11th", ["school_name", "grade", "reading_score"]]
dfr12 = student_data.loc[student_data["grade"] =="12th", ["school_name", "grade", "reading_score"]]

dfr9 = dfr9.groupby(by="school_name", as_index=False)
dfr9 = dfr9.mean()
dfr9 = dfr9.rename(columns = {"reading_score":"9th Grade Reading Score"})
dfr9 = dfr9.rename(columns = {"school_name":"School Name"})

dfr10 = dfr10.groupby(by="school_name", as_index=False)
dfr10 = dfr10.mean()
dfr10 = dfr10.drop(["school_name"], axis=1)
dfr10 = dfr10.rename(columns = {"reading_score":"10th Grade Reading Score"})

dfr11 = dfr11.groupby(by="school_name", as_index=False)
dfr11 = dfr11.mean()
dfr11 = dfr11.drop(["school_name"], axis=1)
dfr11 = dfr11.rename(columns = {"reading_score":"11th Grade Reading Score"})

dfr12 = dfr12.groupby(by="school_name", as_index=False)
dfr12 = dfr12.mean()
dfr12 = dfr12.drop(["school_name"], axis=1)
dfr12 = dfr12.rename(columns = {"reading_score":"12th Grade Reading Score"})

r_s_b_grade = pd.concat([dfr9, dfr10, dfr11, dfr12], axis=1)
r_s_b_grade

,School Name,9th Grade Reading Score,10th Grade Reading Score,11th Grade Reading Score,12th Grade Reading Score
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.